In [19]:
import os
from IPython.display import Image
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

### copy the fuctions from testrun & change the return value to fit other ones's functions

In [20]:
def tail(f, lines=1, _buffer=4098):
    """Tail a file and get X lines from the end"""
    # place holder for the lines found
    lines_found = []

    # block counter will be multiplied by buffer
    # to get the block size from the end
    block_counter = -1

    # loop until we find X lines
    while len(lines_found) < lines:
        try:
            f.seek(block_counter * _buffer, os.SEEK_END)
        except IOError:  # either file is too small, or too many lines requested
            f.seek(0)
            lines_found = f.readlines()
            break

        lines_found = f.readlines()

        # decrement the block counter to get the
        # next X bytes
        block_counter -= 1

    return lines_found[-lines:]

In [21]:
def get_bond_orders(filename):
    """Fetches the bond indices from the end of the JANPA output."""
    f = open(filename)
    lines = tail(f, lines=56)
    f.close()

    split_lines = [line.split() for line in lines[:-9]]

    for line in split_lines:
        del line[0:2]
        line[0] = line[0][:-1]

    n_atoms = len(split_lines)
    bond_orders = np.zeros((n_atoms, n_atoms))

    indices = np.triu_indices(47)

    bond_orders[indices] = [float(item) for sublist in split_lines for item in sublist]
    return bond_orders

In [22]:
def convert_bond_orders(bond_orders):
    """Round bond orders to their nearest bond type"""
    for i in range(bond_orders.shape[0]):
        for j in range(bond_orders.shape[1]):
            if np.isclose(bond_orders[i, j], 1.5, atol=0.15):
                bond_orders[i, j] = 1.5
            else:
                bond_orders[i, j] = int(round(bond_orders[i, j]))
    bonding_matrix = bond_orders
    return bonding_matrix

### test the fuctions

In [23]:
bond_orders = get_bond_orders('output.janpa')
bond_orders

array([[4.0091e+00, 1.4487e+00, 1.2800e-02, ..., 1.0000e-04, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 4.0095e+00, 1.4575e+00, ..., 0.0000e+00, 1.0000e-04,
        1.0000e-04],
       [0.0000e+00, 0.0000e+00, 3.9965e+00, ..., 1.0000e-04, 0.0000e+00,
        0.0000e+00],
       ...,
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 9.4380e-01, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 9.6220e-01,
        6.0000e-04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        9.5850e-01]])

In [24]:
bonding_matrix = convert_bond_orders(bond_orders)
bonding_matrix

array([[4. , 1.5, 0. , ..., 0. , 0. , 0. ],
       [0. , 4. , 1.5, ..., 0. , 0. , 0. ],
       [0. , 0. , 4. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 1. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 1. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 1. ]])

In [34]:
np.savetxt('bonding.txt', bonding_matrix)

### write the test fuctions

In [51]:
bonding_matrix = np.random.random((5,5))*0.1+100
bonding_matrix

array([[100.05572198, 100.06466366, 100.05853934, 100.00802093,
        100.08141151],
       [100.09647313, 100.05992741, 100.09469804, 100.03245655,
        100.02723882],
       [100.04270739, 100.02478694, 100.07029346, 100.08379386,
        100.09130976],
       [100.06427315, 100.07511537, 100.03035856, 100.01236016,
        100.03713383],
       [100.0409027 , 100.00024894, 100.08952878, 100.0824258 ,
        100.02250695]])

In [53]:
def test_convert_bond_orders():
    bond_orders = np.random.random((5,5))*0.01+1.5
    assert convert_bond_orders(bond_orders).shape[0] == convert_bond_orders(bond_orders).shape[1], "the bond orders matrix should be square"
    assert convert_bond_orders(bond_orders)[2][2] == 1.5, "all values about 1.5 should be reduced into exact 1.5"
    bond_orders = np.random.random((5,5))*0.1+10
    assert convert_bond_orders(bond_orders)[2][2] == 10, "other values aren't close to 1.5 should be reduced into intager"
    return

### nosetests

In [62]:
cd ../tests

C:\Users\sxzxc\Desktop\blues\blues\tests


In [63]:
%%cmd
nosetests test_bondorders.py

Microsoft Windows [Version 10.0.17134.48]
(c) 2018 Microsoft Corporation. All rights reserved.

(my-rdkit-env) C:\Users\sxzxc\Desktop\blues\blues\tests>nosetests test_bondorders.py

(my-rdkit-env) C:\Users\sxzxc\Desktop\blues\blues\tests>

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [66]:
np.random.normal(size=(5, 5))

array([[ 0.85540006,  1.95514738, -1.32279911, -1.58292187, -0.67797952],
       [-0.50198462,  0.23821901, -0.32029917, -0.20107096, -0.70446369],
       [-1.67151228,  0.0083155 , -0.67799048, -1.47611445, -1.7940283 ],
       [-1.21026576,  0.17597615,  0.96559679, -0.6023937 ,  1.57442491],
       [ 0.56493612,  1.1226086 , -0.4776737 , -0.64128499, -0.28808397]])